In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Parkinsons_Disease"
cohort = "GSE57475"

# Input paths
in_trait_dir = "../../input/GEO/Parkinsons_Disease"
in_cohort_dir = "../../input/GEO/Parkinsons_Disease/GSE57475"

# Output paths
out_data_file = "../../output/preprocess/Parkinsons_Disease/GSE57475.csv"
out_gene_data_file = "../../output/preprocess/Parkinsons_Disease/gene_data/GSE57475.csv"
out_clinical_data_file = "../../output/preprocess/Parkinsons_Disease/clinical_data/GSE57475.csv"
json_path = "../../output/preprocess/Parkinsons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Association between alpha-synuclein blood transcripts and early Parkinson's disease"
!Series_summary	"Importance. Biological markers of Parkinson’s disease are essential for achieving disease modification. Objective. To determine the association of SNCA blood transcript levels with prevalence of Parkinson’s disease. Background. The SNCA locus is preferentially transcribed in neurons and blood cells. Non-coding genetic variants and neuronal aggregates of α-synuclein protein associate this locus with sporadic Parkinson’s disease and suggest a potential role for abnormal SNCA transcription in the disease mechanism. Here we investigated variation in intracellular SNCA gene expression and SNCA transcript isoform abundance in circulating blood cells of cases with PD and controls in a network of biobanks that represent regional, national, and international populations. Design, Setting, Participants. Three cross-sectional, case-control studies nested in o

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, this dataset appears to be about SNCA gene 
# expression in Parkinson's Disease, so gene expression data is likely available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary:
trait_row = 2  # 'disease state' at index 2
age_row = 0    # 'age' at index 0
gender_row = 1 # 'gender' at index 1

# 2.2 Data Type Conversion
def convert_trait(value):
    if ":" in value:
        value = value.split(":")[1].strip()
    if "PD" in value or "parkinson" in value.lower():
        return 1
    elif "control" in value.lower() or "healthy" in value.lower():
        return 0
    return None

def convert_age(value):
    if ":" in value:
        value = value.split(":")[1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    if ":" in value:
        value = value.split(":")[1].strip()
    if value.upper() == 'F' or value.lower() == 'female':
        return 0
    elif value.upper() == 'M' or value.lower() == 'male':
        return 1
    return None

# 3. Save Metadata
# trait_row is not None, so trait data is available
is_trait_available = trait_row is not None
# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # The sample characteristics dictionary provides the unique values, but
    # we need the actual data in the format expected by geo_select_clinical_features
    
    # Try to load the clinical data file directly
    try:
        # First, try to see if there's a matrix file we can use
        matrix_file = f"{in_cohort_dir}/matrix.txt"
        if os.path.exists(matrix_file):
            # Read the matrix file to get the clinical data
            with open(matrix_file, 'r') as f:
                lines = f.readlines()
            
            # Extract sample characteristics from the matrix file
            sample_chars_data = {}
            for i, line in enumerate(lines):
                if line.startswith("!Sample_characteristics_ch"):
                    parts = line.strip().split('\t')
                    if len(parts) >= 2:
                        key = parts[0].replace("!Sample_characteristics_ch", "").strip()
                        value = parts[1]
                        if key not in sample_chars_data:
                            sample_chars_data[key] = []
                        sample_chars_data[key].append(value)
            
            # Create a DataFrame suitable for geo_select_clinical_features
            clinical_data = pd.DataFrame(sample_chars_data)
        else:
            # If matrix file doesn't exist, create a minimal DataFrame with the information we have
            # The function expects a DataFrame where each column corresponds to a feature
            # and each row corresponds to a sample
            sample_chars = {
                0: ['age: 62', 'age: 54', 'age: 58', 'age: 76', 'age: 52', 'age: 72', 'age: 78', 'age: 61', 'age: 67', 'age: 68', 
                    'age: 84', 'age: 65', 'age: 86', 'age: 49', 'age: 71', 'age: 69', 'age: 64', 'age: 55', 'age: 60', 'age: 56', 
                    'age: 53', 'age: 63', 'age: 80', 'age: 45', 'age: 75', 'age: 66', 'age: 51', 'age: 70', 'age: 57', 'age: 39'], 
                1: ['gender: M', 'gender: F'], 
                2: ['disease state: Dopamine Transporter Imaging (DAT)-confirmed PD', 'disease state: healthy control']
            }
            
            # Since we only have unique values and not the full dataset,
            # we'll create a dummy index for the clinical data DataFrame that the function can process
            clinical_data = pd.DataFrame({str(k): v for k, v in sample_chars.items()}).T
    
    except Exception as e:
        print(f"Error loading clinical data: {e}")
        # Create an empty DataFrame as fallback
        clinical_data = pd.DataFrame()
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the processed clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Error loading clinical data: All arrays must be of the same length
Preview of selected clinical features:
{}
Clinical data saved to ../../output/preprocess/Parkinsons_Disease/clinical_data/GSE57475.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Gene identifiers analysis 
# The identifiers starting with "ILMN_" are Illumina probe IDs, not human gene symbols
# These are microarray probe identifiers from Illumina platform and need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174', 'ILMN_1796063', 'ILMN_1811966'], 'nuID': ['rp13_p1x6D80lNLk3c', 'NEX0oqCV8.er4HVfU4', 'KyqQynMZxJcruyylEU', 'xXl7eXuF7sbPEp.KFI', '9ckqJrioiaej9_ajeQ'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'Unigene', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'Transcript': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'ILMN_Gene': ['LOC23117', 'HS.575038', 'FCGR2B', 'TRIM44', 'LOC653895'], 'Source_Reference_ID': ['XM_933824.1', 'Hs.575038', 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'RefSeq_ID': ['XM_933824.1', nan, 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'Unigene_ID': [nan, 'Hs.575038', nan, nan, nan], 'Entrez_Gene_ID': [23117.0, nan, 2213.0, 54765.0, 653895.0], 'GI': [89040007.0, 10437021.0, 88952550.0, 29029528.0, 89033487.0], 'Access

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for mapping
# From the gene annotation preview, we can see:
# - 'ID' column contains the Illumina probe identifiers (e.g., ILMN_1725881)
# - 'Symbol' column contains the gene symbols (e.g., LOC23117, FCGR2B)

prob_col = 'ID'
gene_col = 'Symbol'

# 2. Get the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print the shape of the resulting gene expression dataframe
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 5 gene symbols after mapping:")
print(gene_data.index[:5])


Gene expression data shape after mapping: (19106, 142)
First 5 gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A26A1', 'A26B1', 'A26C1B'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols from the already mapped gene expression data from Step 6
# Apply normalization to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {normalized_gene_data.shape}")
print("First 5 normalized gene symbols:")
print(normalized_gene_data.index[:5])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load clinical data that was extracted in step 2
# We need to re-extract the clinical features since the previous attempt encountered an error
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Extract clinical features with the parameters defined in step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=2,  # From step 2
    convert_trait=convert_trait,
    age_row=0,    # From step 2
    convert_age=convert_age,
    gender_row=1, # From step 2
    convert_gender=convert_gender
)

print("Clinical data shape:", selected_clinical_df.shape)
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the processed clinical data (rewriting potentially empty file from step 2)
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
# Transpose clinical data to have samples as rows
clinical_data_t = selected_clinical_df.T
print(f"Transposed clinical data shape: {clinical_data_t.shape}")

# Transpose genetic data to have samples as rows
genetic_data_t = normalized_gene_data.T
print(f"Transposed genetic data shape: {genetic_data_t.shape}")

# Check for overlapping samples
clinical_samples = set(clinical_data_t.index)
genetic_samples = set(genetic_data_t.index)
common_samples = clinical_samples.intersection(genetic_samples)
print(f"Number of common samples: {len(common_samples)}")

# Filter to keep only common samples
clinical_data_t = clinical_data_t.loc[list(common_samples)]
genetic_data_t = genetic_data_t.loc[list(common_samples)]

# Merge clinical and genetic data
linked_data = pd.concat([clinical_data_t, genetic_data_t], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Data shape after removing biased features: {linked_data.shape}")
print(f"Is the trait biased: {is_biased}")

# 6. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from blood samples of PD patients and controls."
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable. Linked data was not saved.")

Gene expression data shape after normalization: (18312, 142)
First 5 normalized gene symbols:
Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Parkinsons_Disease/gene_data/GSE57475.csv


Clinical data shape: (3, 142)
Clinical data preview:
{'GSM1383302': [1.0, 62.0, 1.0], 'GSM1383303': [1.0, 54.0, 1.0], 'GSM1383304': [1.0, 58.0, 1.0], 'GSM1383305': [1.0, 76.0, 1.0], 'GSM1383306': [1.0, 52.0, 0.0], 'GSM1383307': [1.0, 52.0, 0.0], 'GSM1383308': [1.0, 72.0, 0.0], 'GSM1383309': [1.0, 54.0, 0.0], 'GSM1383310': [1.0, 78.0, 1.0], 'GSM1383311': [1.0, 61.0, 0.0], 'GSM1383312': [1.0, 67.0, 1.0], 'GSM1383313': [1.0, 68.0, 1.0], 'GSM1383314': [1.0, 76.0, 1.0], 'GSM1383315': [1.0, 84.0, 1.0], 'GSM1383316': [1.0, 65.0, 0.0], 'GSM1383317': [1.0, 58.0, 0.0], 'GSM1383318': [1.0, 86.0, 0.0], 'GSM1383319': [1.0, 49.0, 0.0], 'GSM1383320': [1.0, 72.0, 0.0], 'GSM1383321': [1.0, 71.0, 1.0], 'GSM1383322': [1.0, 69.0, 0.0], 'GSM1383323': [1.0, 64.0, 1.0], 'GSM1383324': [1.0, 55.0, 0.0], 'GSM1383325': [1.0, 69.0, 0.0], 'GSM1383326': [1.0, 60.0, 1.0], 'GSM1383327': [1.0, 56.0, 1.0], 'GSM1383328': [1.0, 60.0, 1.0], 'GSM1383329': [1.0, 61.0, 1.0], 'GSM1383330': [1.0, 53.0, 0.0], 'GSM1383331': [1.0

Data shape after handling missing values: (142, 18315)
For the feature 'Parkinsons_Disease', the least common label is '0.0' with 49 occurrences. This represents 34.51% of the dataset.
The distribution of the feature 'Parkinsons_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 55.0
  50% (Median): 61.0
  75%: 69.0
Min: 39.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 54 occurrences. This represents 38.03% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data shape after removing biased features: (142, 18315)
Is the trait biased: False


Linked data saved to ../../output/preprocess/Parkinsons_Disease/GSE57475.csv
